### Question 2

In [ ]:
# importing libraries
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import ast
import string

pd.set_option('display.max_colwidth', None)

In [ ]:
# raeding data from Q1
data = pd.read_csv("~/Desktop/IR/Assignments/Assignment1/final_df.csv")

In [ ]:
# making word corpus of all the words in all the documents
word_corpus = set()
for content in data['Contents of file after Preprocessing']:
    content = ast.literal_eval(content)
    for token in content:
        word_corpus.add(token)

In [ ]:
# renaming column
data.rename(columns={'Doc Id':'Doc_Id'}, inplace=True)

In [ ]:
# creating unigram inverted index
inverted_index = {}

for word in word_corpus:
    for docId in data["Doc_Id"]:
        if(word in ast.literal_eval(data.loc[data["Doc_Id"] == docId, "Contents of file after Preprocessing"].iloc[0])):
            if word not in inverted_index:
                inverted_index[word] = []
            if word in inverted_index and docId not in inverted_index[word]:
                inverted_index[word].append(docId)

In [ ]:
# using pickle to load inverted index
import pickle
file = open("/Users/amritaaash/Desktop/inverted_index", 'rb')
inverted_index = pickle.load(file)
for key,val in inverted_index.items():
    val.sort()
file.close()

In [ ]:
# function to pre-process query: lowercase,tokenize,remove stop_words,punctuation and blank spaces
punctuations = string.punctuation
punc = [ p for p in punctuations]

def preprocess(query):
    query = query.lower()
    tokens = word_tokenize(query)
    remove_stop = [t for t in tokens if not t in stopwords.words("english")]
    remove_punc = [t for t in remove_stop if t not in punc]
    res = [ t.strip() for t in remove_punc if len(t.strip()) != 0]
    return res

In [ ]:
# function to perform operation on two posting list
def operate(list1, list2, operation):
    if operation == "OR":
        i = 0
        j = 0
        res_list = []
        comp = 0
        while (i < len(list1) and j < len(list2)):
            if (int(list1[i][9:]) < int(list2[j][9:])):
                res_list.append(list1[i])
                i += 1
            elif (int(list2[j][9:]) < int(list1[i][9:])):
                res_list.append(list2[j])
                j += 1
            else:
                res_list.append(list1[i])
                i += 1
                j += 1
            comp += 1
        while (i < len(list1)):
            res_list.append(list1[i])
            i += 1
        while (j < len(list2)):
            res_list.append(list2[j])
            j += 1
        return (res_list, comp)
    elif operation == "AND":
        i = 0
        j = 0
        res_list = []
        comp = 0
        while (i < len(list1) and j < len(list2)):
            if (int(list1[i][9:]) < int(list2[j][9:])):
                i += 1
            elif (int(list2[j][9:]) < int(list1[i][9:])):
                j += 1
            else:
                res_list.append(list1[i])
                i += 1
                j += 1
            comp += 1
        return (res_list, comp) 
    elif operation == "AND NOT":
        i = 0
        j = 0
        res_list = []
        comp = 0
        while (i < len(list1) and j < len(list2)):
            if (int(list1[i][9:]) > int(list2[j][9:])):
                j += 1
            elif (int(list1[i][9:]) < int(list2[j][9:])):
                res_list.append(list1[i])
                i += 1
            else:
                i += 1
                j += 1
            comp += 1
        return(res_list, comp)             

    elif operation == "OR NOT":
        U = set(data["Doc_Id"])
        set_list2 = set(list2)
        list2_comp = U.difference(set_list2)
        list2_comp = list(list2_comp)
        list2_comp.sort()
        res_list, comp = operate(list1, list2_comp, "OR")
        return (res_list, comp)
    else:
        print(f"Wrong operation provided, {operation} not supported")
        return (0, 0)


# function to print the desired outputs of given query and operation
def retrieve(query, operations,query_num):
    query = preprocess(query)

    operations = operations.split(',')
    operations = [t.strip() for t in operations]
    a = len(query)
    b = len(operations)
    if ((a-b) == 1):
        pass
    else:
        print("query not compatible with the provided no. of operations")
        return
    query_docs = []
    for q in query:
        try:
            query_docs.append(inverted_index[q])
        except:
            query_docs.append([])

    total_compar = 0
    for i in range(len(operations)):
        doc_list1 = query_docs[0]
        doc_list2 = query_docs[1]
        doc_list_res, no_of_compar = operate(doc_list1, doc_list2, operations[i])
        if (doc_list_res == 0):
            return
        total_compar += no_of_compar
        del query_docs[0]
        del query_docs[0]
        query_docs.insert(0, doc_list_res)
    
    output_str = " "
    i = 0
    j = 0
    while(j<len(operations)):
        output_str += query[i]
        output_str += " "
        output_str += operations[j]
        output_str += " "
        i += 1
        j += 1
    output_str += query[i]

    print(f"Query {query_num}: {output_str}")

    print(f"Number of documents retrieved for query {query_num}: {len(doc_list_res)}")
    
    print(f"Names of documents retrieved for query {query_num}: ",end = "")
    for d in doc_list_res:
        print(d + " ",end = " ")
    print()

    print(f"Number of comparisons required for query {query_num}: {total_compar}")


In [ ]:
n = int(input("Enter number of queries to execute: "))
for i in range(n):
    query = input()
    operation = input()
    retrieve(query,operation,i+1)